![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [130]:
df = pd.read_csv('bank_marketing.csv')
columns = df.columns
for col in columns:
    print(col)
    print(df[col].value_counts())

client_id
client_id
0        1
27512    1
27454    1
27455    1
27456    1
        ..
13730    1
13731    1
13732    1
13733    1
41187    1
Name: count, Length: 41188, dtype: int64
age
age
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
89       2
91       2
94       1
87       1
95       1
Name: count, Length: 78, dtype: int64
job
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64
marital
marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64
education
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illi

In [131]:
# Splitting the file into three different dataframes

client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
campaign = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']] 

print(client.head())
print(campaign.head())
print(economics.head()) 

   client_id  age        job  marital    education credit_default mortgage
0          0   56  housemaid  married     basic.4y             no       no
1          1   57   services  married  high.school        unknown       no
2          2   37   services  married  high.school             no      yes
3          3   40     admin.  married     basic.6y             no       no
4          4   56   services  married  high.school             no       no
   client_id  number_contacts  contact_duration  previous_campaign_contacts  \
0          0                1               261                           0   
1          1                1               149                           0   
2          2                1               226                           0   
3          3                1               151                           0   
4          4                1               307                           0   

  previous_outcome campaign_outcome  
0      nonexistent               no  

In [132]:
# Cleaning the client columns as pper the requirements

client['job'] = client['job'].astype('str').str.replace('.', '_')

client['education'] = client['education'].astype('str').str.replace('.', '_')
client['education'] = client['education'].str.replace('unknown', 'np.Nan')
client['education'] = client['education'].astype('object')

for col in ['credit_default', 'mortgage']:
    client[col] = np.where(client[col] == 'yes', 1, 0)
    client[col] = client[col].astype('bool')
    print(client[col].value_counts())


credit_default
False    41185
True         3
Name: count, dtype: int64
mortgage
True     21576
False    19612
Name: count, dtype: int64


C:\Users\Owner\AppData\Local\Temp\ipykernel_20704\556708519.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['job'] = client['job'].astype('str').str.replace('.', '_')
C:\Users\Owner\AppData\Local\Temp\ipykernel_20704\556708519.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client['education'] = client['education'].astype('str').str.replace('.', '_')
C:\Users\Owner\AppData\Local\Temp\ipykernel_20704\556708519.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [133]:
# Verifyin the data types for the clinet column
for col in client.columns:
    print(client[col].dtypes)

campaign['previous_outcome'].value_counts()

int64
int64
object
object
object
bool
bool


previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64

In [134]:
# Data cleaning for the campaign column
for col1 in ['previous_outcome', 'campaign_outcome']:
    if col1 == 'previous_outcome':
        campaign[col1] =campaign[col1].astype('str')
        campaign[col1] = np.where(campaign[col1] == 'success', '1', '0')
    else:
        campaign[col1] = np.where(campaign[col1] == 'yes', 1, 0)
    campaign[col1] = campaign[col1].astype('bool')
    print(campaign[col1].value_counts())


previous_outcome
True    41188
Name: count, dtype: int64
campaign_outcome
False    36548
True      4640
Name: count, dtype: int64


C:\Users\Owner\AppData\Local\Temp\ipykernel_20704\3464738590.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign[col1] =campaign[col1].astype('str')
C:\Users\Owner\AppData\Local\Temp\ipykernel_20704\3464738590.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign[col1] = np.where(campaign[col1] == 'success', '1', '0')
C:\Users\Owner\AppData\Local\Temp\ipykernel_20704\3464738590.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro